<a href="https://colab.research.google.com/github/caendeith/mlBTC/blob/main/mlBTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go

In [7]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "btcusd_1-min_data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "mczielinski/bitcoin-historical-data",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipython-input-7-2705815040.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:       Timestamp  Open  High   Low  Close  Volume
0  1.325412e+09  4.58  4.58  4.58   4.58     0.0
1  1.325412e+09  4.58  4.58  4.58   4.58     0.0
2  1.325412e+09  4.58  4.58  4.58   4.58     0.0
3  1.325412e+09  4.58  4.58  4.58   4.58     0.0
4  1.325412e+09  4.58  4.58  4.58   4.58     0.0


In [8]:
# Преобразование Timestamp в дату
df['Date'] = pd.to_datetime(df['Timestamp'], unit='s')

# Оставляем только Date и Close
df = df[['Date', 'Close']].sort_values(by='Date').reset_index(drop=True)

# Агрегация по дням (если несколько записей за день)
df_daily = df.resample('D', on='Date').last().reset_index()

# Берём последние 2000 дней
df_last = df_daily.tail(2000).copy()
print("Последние 2000 дней загружены")
print(df_last.head())

Последние 2000 дней загружены
           Date    Close
2925 2020-01-04  7291.27
2926 2020-01-05  7430.83
2927 2020-01-06  7525.46
2928 2020-01-07  8050.00
2929 2020-01-08  8051.10


In [27]:
# Подготовка ряда
ts = df_last.set_index('Date')['Close']

# Разделим данные на train и test
train_size = int(len(ts) * 0.9)
train, test = ts[:train_size], ts[train_size:]

# Обучаем модель на тренировочных данных
model_train = ARIMA(train, order=(4, 1, 2))
arima_train_model = model_train.fit()

# Прогнозируем на длину тестовой выборки
forecast_test = arima_train_model.forecast(steps=len(test))

# Сравниваем прогноз и реальные значения
comparison = pd.DataFrame({
    'Тестовые данные': test.values,
    'Прогноз ARIMA': forecast_test
}, index=test.index)

print("\nСравнение прогноза и реальных значений:")
print(comparison.head())

# Оценка точности
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(test, forecast_test)
mae = mean_absolute_error(test, forecast_test)
print(f"\nMSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.




Сравнение прогноза и реальных значений:
            Тестовые данные  Прогноз ARIMA
Date                                      
2024-12-08         101204.0  100334.255031
2024-12-09          97468.0  100326.671486
2024-12-10          96648.0  100569.570581
2024-12-11         101201.0  100707.638804
2024-12-12         100068.0  100892.900524

MSE: 163862156.34
MAE: 9736.72


In [28]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=train.index, y=train.values, name='Обучающие данные'))
fig.add_trace(go.Scatter(x=test.index, y=test.values, name='Тестовые данные'))
fig.add_trace(go.Scatter(x=test.index, y=forecast_test.values, name='Прогноз ARIMA'))

fig.update_layout(title='Сравнение реальной и прогнозной цены (ARIMA)', template='plotly_white')
fig.show()